
# Multi-objective Bayesian Optimization


TNK function
$n=2$ variables:
$x_i \in [0, \pi], i=1,2$

Objectives:
- $f_i(x) = x_i$

Constraints:
- $g_1(x) = -x_1^2 -x_2^2 + 1 + 0.1 \cos\left(16 \arctan \frac{x_1}{x_2}\right) \le 0$
- $g_2(x) = (x_1 - 1/2)^2 + (x_2-1/2)^2 \le 0.5$

In [7]:
%reset -f
import numpy as np

import torch
from xopt import XoptBase, Evaluator
from xopt.generators.bayesian.mobo import MOBOGenerator, MOBOOptions
from xopt.resources.test_functions.tnk import evaluate_TNK, tnk_vocs

import pandas as pd

evaluator = Evaluator(evaluate_TNK)
options = MOBOOptions()
options.dict()

{'name': None,
 'version': None,
 'optim': {'num_restarts': 5, 'raw_samples': 20, 'sequential': True},
 'acq': {'objective': None,
  'sampler': SobolQMCNormalSampler(),
  'ref_point': None,
  'use_data_as_reference': True},
 'model': {'input_transform': None, 'outcome_transform': None},
 'tkwargs': {'device': 'cpu', 'dtype': torch.float64},
 'n_initial': 3,
 'proximal_lengthscales': None}

In [8]:

generator = MOBOGenerator(tnk_vocs, options=options)
X = XoptBase(generator=generator, evaluator=evaluator, vocs=tnk_vocs)
X.step()


In [9]:
print(X.data)

None


In [10]:
for i in range(50):
    print(f'step {i}')
    X.step()

step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10


KeyboardInterrupt: 

In [ ]:
from xopt.generators.bayesian.utils import feasibility
from matplotlib import pyplot as plt# plot model predictions
bounds = tnk_vocs.bounds
history = X.data
model = X.generator.train_model(history)


n = 100
x = torch.linspace(*bounds.T[0],n)
y = torch.linspace(*bounds.T[1],n)
xx,yy = torch.meshgrid(x,y)
pts = torch.hstack([ele.reshape(-1,1) for ele in (xx,yy)]).double()

with torch.no_grad():
    acq_pts = pts.unsqueeze(1)
    acq = X.generator.get_acquisition(model)(acq_pts)

    fig, ax = plt.subplots(figsize=(8,8))
    c = ax.pcolor(xx,yy,acq.reshape(n,n))
    fig.colorbar(c)
    ax.set_title('Acquisition function')

    ax.plot(*history[["x1","x2"]].to_numpy().T,'.C1')

    feas = feasibility(
        pts.unsqueeze(1),
        model,
        generator.options.acq.sampler,
        tnk_vocs
    ).flatten()

fig2, ax2 = plt.subplots(figsize=(8,8))
c2 = ax2.pcolor(xx,yy,feas.reshape(n,n))
fig2.colorbar(c2)
ax2.set_title('Feasibility')

ax.plot(*history[["x1","x2"]].to_numpy().T,'.C1')


theta = np.linspace(0,np.pi/2)
r = np.sqrt(1 + 0.1*np.cos(16*theta))
x_1 = r*np.sin(theta)
x_2_lower = r*np.cos(theta)
x_2_upper = (0.5 - (x_1 - 0.5)**2)**0.5 + 0.5

z = np.zeros_like(x_1)
ax.plot(x_1, x_2_lower,'r')
ax2.plot(x_1, x_2_lower,'r')
ax2.plot(*history[["x1","x2"]].to_numpy().T,'.C1')
ax2.set_xlim(0, 2)
ax2.set_ylim(0, 2)
ax.set_xlabel('x1')
ax.set_ylabel('x2')

ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax2.set_aspect('equal')

In [ ]:

fig,ax = plt.subplots()

theta = np.linspace(0,np.pi/2)
r = np.sqrt(1 + 0.1*np.cos(16*theta))
x_1 = r*np.sin(theta)
x_2_lower = r*np.cos(theta)
x_2_upper = (0.5 - (x_1 - 0.5)**2)**0.5 + 0.5

z = np.zeros_like(x_1)

#ax2.plot(x_1, x_2_lower,'r')
ax.fill_between(x_1, z, x_2_lower, fc='white')
circle = plt.Circle((0.5, 0.5), 0.5**0.5,
                    color='r', alpha=0.25, zorder=0,
                    label='Valid Region')
ax.add_patch(circle)
history = pd.concat([X.data, tnk_vocs.feasibility_data(X.data)], axis=1, ignore_index=False)

ax.plot(*history[["x1","x2"]][history["feasible"]].to_numpy().T,'.C1')
ax.plot(*history[["x1","x2"]][~history["feasible"]].to_numpy().T,'.C2')

ax.set_xlim(0, 2)
ax.set_ylim(0, 2)
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_aspect('equal')